### Loading Kaggle dependencies

In [0]:
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 110.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

### Kaggle API 

In [0]:
import os
os.environ["KAGGLE_API_TOKEN"] = "KGAT_accf5044a08d73b8271e183550e8edea"

print("Kaggle credentials configured!")

Kaggle credentials configured!


In [0]:
!kaggle competitions list

ref                                                                              deadline             category          reward  teamCount  userHasEntered  
-------------------------------------------------------------------------------  -------------------  ---------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-3    2026-04-15 23:59:00  Featured   2,207,152 Usd       1242           False  
https://www.kaggle.com/competitions/passenger-screening-algorithm-challenge      2017-12-15 23:59:00  Featured   1,500,000 Usd        518           False  
https://www.kaggle.com/competitions/zillow-prize-1                               2018-01-10 15:59:00  Featured   1,200,000 Usd       3770           False  
https://www.kaggle.com/competitions/data-science-bowl-2017                       2017-04-12 23:59:00  Featured   1,000,000 Usd       1972           False  
https://www.kaggle.com/competitions/vesuvius-challenge-ink-detec

### Creating Schema

In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce_idc
""")

DataFrame[]

### Create Volume to upload files

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce_idc.ecommerce_data_idc
""")

DataFrame[]

In [0]:
%sh
cd /Volumes/workspace/ecommerce_idc/ecommerce_data_idc
kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors


100%|██████████| 4.29G/4.29G [00:27<00:00, 169MB/s]


In [0]:
%sh
cd /Volumes/workspace/ecommerce_idc/ecommerce_data_idc
unzip -o ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            
total 18G
-rwxrwxrwx 1 spark-ac0ba1d2-227a-4daa-b01a-4b nogroup 8.4G Jan  9 00:29 2019-Nov.csv
-rwxrwxrwx 1 spark-ac0ba1d2-227a-4daa-b01a-4b nogroup 5.3G Jan  9 00:31 2019-Oct.csv
-rwxrwxrwx 1 spark-ac0ba1d2-227a-4daa-b01a-4b nogroup 4.3G Jan  9 00:22 ecommerce-behavior-data-from-multi-category-store.zip


In [0]:
%sh
cd /Volumes/workspace/ecommerce_idc/ecommerce_data_idc
rm -f ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

total 14G
-rwxrwxrwx 1 nobody nogroup 8.4G Jan  9 00:29 2019-Nov.csv
-rwxrwxrwx 1 nobody nogroup 5.3G Jan  9 00:31 2019-Oct.csv


In [0]:
%restart_python

### Loading Oct and Nov ecommerce Data

In [0]:
df_nov = spark.read.csv("/Volumes/workspace/ecommerce_idc/ecommerce_data_idc/2019-Nov.csv",header=True,inferSchema=True)

In [0]:
df_oct = spark.read.csv("/Volumes/workspace/ecommerce_idc/ecommerce_data_idc/2019-Oct.csv",header=True,inferSchema=True)

In [0]:
print(f"October 2019 - Total Events: {df_oct.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
df_oct.printSchema()

October 2019 - Total Events: 42,448,764

SCHEMA:
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
print("\n" + "="*60)
print("SAMPLE DATA (First 5 rows):")
print("="*60)
display(df_oct.head(5))


SAMPLE DATA (First 5 rows):


event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-01T00:00:00.000Z,view,44600062,2103807459595387724,null,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
2019-10-01T00:00:00.000Z,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.2,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2019-10-01T00:00:01.000Z,view,17200506,2053013559792632471,furniture.living_room.sofa,null,543.1,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
2019-10-01T00:00:01.000Z,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
2019-10-01T00:00:04.000Z,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
